In [ ]:
import numpy as np
rng = np.random.default_rng(0)


In [ ]:
# Helper: generate random pointer tables T_j : [m] -> [m] for j=1..k

def generate_tables(m: int, k: int, rng: np.random.Generator):
    # Each T_j is represented as a length-m array with entries in [0, m-1]
    return [rng.integers(low=0, high=m, size=m, dtype=np.int64) for _ in range(k)]


def compose_pointer(T_list, s: int):
    u = s
    for T in T_list:
        u = int(T[u])
    return u


def ceil_log2(m: int) -> int:
    return int(np.ceil(np.log2(max(2, m))))


In [ ]:
# Parameters and empirical estimation of fooling family size
m_values = [8, 16, 32, 64, 128]
k = 5  # depth parameter (k >= 2)
c = 1.0  # budget constant
alpha_target = 0.9

results = []
for m in m_values:
    # Fix T1..T_{k-1}, vary Tk and b on a large subset S (|S| >= 0.9 m)
    T_prefix = generate_tables(m, k-1, rng)
    s = int(rng.integers(low=0, high=m))
    # Construct S of size floor(0.9 m)
    S_size = int(np.floor(0.9 * m))
    alpha_emp = S_size / m
    # Encoding size: n = k*m*ceil(log2 m) + m
    n = k * m * ceil_log2(m) + m
    # Empirical proxy for log2 M using last-layer degrees of freedom over S
    # M ≈ 2^{alpha*m} => log2 M ≈ alpha*m
    log2_M_est = alpha_emp * m
    # Lower bound proxy: T ≥ log M / B(k-1,n) with B = c*(k-1)*log2(n)
    denom = c * (k - 1) * np.log2(max(2, n))
    T_lb_est = log2_M_est / denom
    results.append({
        'm': m,
        'n': n,
        'alpha_emp': alpha_emp,
        'log2_M_est': log2_M_est,
        'T_lb_est': T_lb_est
    })

# Display a compact table
print("m\talpha_emp\tlog2 M est (bits)\tT_lb_est")
for r in results:
    print(f"{r['m']}\t{r['alpha_emp']:.3f}\t{r['log2_M_est']:.2f}\t{r['T_lb_est']:.4f}")


In [ ]:
### Empirical trend: m (table size) vs log₂ M (distinguishable instances, bits)

We fix $T_1,\ldots,T_{k-1}$ and vary the last layer $(T_k, b)$ over a large subset $S \subseteq [m]$ with $|S| \ge 0.9m$. As a proxy, we take $\log_2 M \approx \alpha m$ with $\alpha = |S|/m$. The theoretical lower bound is $T \gtrsim (\log M) / (c (k-1) \log n)$. 


In [ ]:
import matplotlib.pyplot as plt

ms = np.array([r['m'] for r in results], dtype=float)
log2M = np.array([r['log2_M_est'] for r in results], dtype=float)
alpha_emp = np.array([r['alpha_emp'] for r in results], dtype=float)

alpha_line = float(alpha_target)  # explicit

plt.figure(figsize=(6.4, 4.2))
plt.plot(ms, log2M, 'o-', label='empirical log2 M ≈ α·m')
plt.plot(ms, alpha_line * ms, '--', label=f'theoretical α·m (α={alpha_line:.2f})')
plt.xlabel('m')
plt.ylabel('α·m (theoretical)')
plt.title('Pointer-Chase L_k: α·m vs m')
plt.grid(True, alpha=0.3)
plt.legend()
# Save plot directly for paper inclusion
plt.savefig('../paper/fig/lk_logM.png', dpi=300, bbox_inches='tight')
plt.close()

# Report alpha and entropy comparison vs budget
for r in results:
    n = r['n']
    m = r['m']
    log2M_emp = r['log2_M_est']
    denom = c * (k - 1) * np.log2(max(2, n))
    print(f"m={m}, alpha_emp={r['alpha_emp']:.3f}, alpha_line={alpha_line:.2f}, T_lb_est={log2M_emp/denom:.4f}")

print('Saved plot to ../paper/fig/lk_logM.png')


In [ ]:
# Entropy analysis: actual vs theoretical α·m bound
for r in results:
    alpha_emp = r['alpha_emp']
    m = r['m']
    log2M_emp = r['log2_M_est']
    log2M_theory = alpha_target * m
    print(f"m={m}: alpha_emp={alpha_emp:.3f}, log2M_emp={log2M_emp:.2f}, theory={log2M_theory:.2f}")
